In [1]:
#  Code adapted from: https://github.com/aladdinpersson/Machine-Learning-Collection/blob/ac5dcd03a40a08a8af7e1a67ade37f28cf88db43/ML/Pytorch/GANs/2.%20DCGAN/train.py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as tfms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import os
import numpy as np
import torch
import torch.nn as nn
import BNN

In [2]:
network = BNN.Network('cuda')

In [3]:
# Calculates noise matrix dimensions required to get desired output dimensions
def noise_size(map_size):
    # For each layer, new = ((prev-1) * stride) - (2*padding) + kernel_size

    # Inverse size of conv 5
    noise_dims = map_size / 2

    # Inverse size of conv 4
    noise_dims = noise_dims / 2
    
    # Inverse size of conv 3
    noise_dims = noise_dims / 2
    
    # Inverse size of conv 2
    noise_dims = noise_dims / 2
    
    # Inverse size of conv 1
    noise_dims = (noise_dims/2) - 1

    return noise_dims


In [4]:
# def test():
    # N, in_channels, H, W = 8, 3, 64, 64
    # noise_dim = 100
    # x = torch.randn((N, in_channels, H, W))
    # disc = Discriminator(in_channels, 8)
    # assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    # gen = Generator(noise_dim, in_channels, 8)
    # z = torch.randn((N, noise_dim, 1, 1))
    # assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"

In [5]:
# test()

In [6]:
# Need to override __init__, __len__, __getitem__
# as per datasets requirement
class PathsDataset(torch.utils.data.Dataset):
    # init the dataset, shape = L x W
    def __init__(self, path, transform=None, shape = (100,100)):
        print("Loading paths dataset...")
        # Read in path files
        # Convert to x by y np arrays
        # add the np arrays to a list
        # set self.transform and self.data
        self.paths = [] # create a list to hold all paths read from file
        for filename in os.listdir(path):
            with open(os.path.join(path, filename), 'r') as f: # open in readonly mode
                self.flat_path = np.loadtxt(f) # load in the flat path from file
                self.path = np.asarray(self.flat_path, dtype=int).reshape(len(self.flat_path)//2,2) #unflatten the path from the file
                
                # xvales which to interpolate on
                # want to interpolate on xvalues from the min xval in the path to the largest xval in the path
                # self.xvals = np.linspace(int(min(self.path[:,0])), int(max(self.path[:,0])), int(max(self.path[:,0])-min(self.path[:,0])))
                # self.xvals = self.xvals.astype(int)

                # interpolate for all xvals using the paths from file's x and y values
                # self.interp_path = np.interp(self.xvals, self.path[:,0], self.path[:,1])
                # self.interp_path = np.array(self.interp_path).astype(int)

                # create a LxW matrix where all the values where path is equal to 1
                # self.path_matrix = np.zeros(shape)
                # self.path_matrix[self.interp_path, self.xvals] = 1
                self.path_matrix = self.convert_path(shape, self.path)
                

                self.paths.append(self.path_matrix) # add the path to paths list
        self.transform = transform
        print("Done!")

    def convert_path(self, map_dim, path):
        path_mat = np.zeros(map_dim, dtype=float)

        # Make the path continuous
        for i in range(path.shape[0] - 1):
            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            if (x1 < x2):
                x_dir = 1
            else:
                x_dir = -1

            if (y1 < y2):
                y_dir = 1
            else:
                y_dir = -1

            # Determine y from x
            if x2-x1 != 0:
                m = (y2-y1)/(x2-x1)
                while x != x2:
                    y = round(m*(x-x1) + y1)
                    path_mat[y,x] = 1
                    x += x_dir
            else:
                while x != x2:
                    path_mat[y1,x] = 1
                    x += x_dir


            x = path[i,0]
            x1 = path[i,0]
            x2 = path[i+1,0]

            y = path[i,1]
            y1 = path[i,1]
            y2 = path[i+1,1]

            # Determine x from y
            if y2-y1 != 0:
                m = (x2-x1)/(y2-y1)
                while y != y2:
                    x = round(m*(y-y1) + x1)
                    path_mat[y,x] = 1
                    y += y_dir
            else:
                while y != y2:
                    path_mat[y,x1] = 1
                    y += y_dir
            
        path_mat[path[path.shape[0]-1,1], path[path.shape[0]-1,0]] = 1     # Include the last point in the path

        return path_mat

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        # print("getitem")
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # imagePath = self.paths_file + "/" + self.data['Image_path'][idx]
        # image = sk.imread(imagePath)
        # label = self.data['Condition'][idx]
        # image = Image.fromarray(image)

        # if self.sourceTransform:
        #     image = self.sourceTransform(image)
        x = np.float32(self.paths[idx])

        if self.transform:
            x = self.transform(x).cuda()
            

        return x

        #return image, label

In [7]:
# Hyperparameters etc.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE_DISC = 0.0002  # could also use two lrs, one for gen and one for disc
LEARNING_RATE_GEN = 0.0002  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 10
IMAGE_SIZE = 256
CHANNELS_IMG = 1
NOISE_DIM = 100
NUM_EPOCHS = 50
FEATURES_DISC = 256
FEATURES_GEN = 256


In [8]:
transforms = tfms.Compose(
    [
        tfms.ToPILImage(),
        tfms.ToTensor(),
        tfms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)


In [9]:
# If you train on MNIST, remember to set channels_img to 1
# dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms,
#                        download=True)

In [10]:
# comment mnist above and uncomment below if train on CelebA
#dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)

dataset = PathsDataset(path = "./env/map_64x64/", shape = (64,64), transform=transforms)

dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)



Loading paths dataset...
Done!


In [11]:
dataset[0]

tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         ...,
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.],
         [-1., -1., -1.,  ..., -1., -1., -1.]]], device='cuda:0')

In [12]:
# opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE_GEN, betas=(0.5, 0.999))
# opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE_DISC, betas=(0.5, 0.999))
# criterion = nn.BCELoss()

In [13]:
# fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
# writer_real = SummaryWriter(f"logs/real")
# writer_fake = SummaryWriter(f"logs/fake")
# step = 0

In [14]:
# gen.train()
# disc.train()

In [15]:
network.train(dataloader, epochs=NUM_EPOCHS)
# for epoch in range(NUM_EPOCHS):
#     # Target labels not needed! <3 unsupervised
#     # for batch_idx, (real, _) in enumerate(dataloader):
#     for batch_idx, real in enumerate(dataloader):
#         real = real.to(device)
#         noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
#         fake = gen(noise)

#         ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
#         disc_real = disc(real.float()).reshape(-1)
#         loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
#         disc_fake = disc(fake.detach()).reshape(-1)
#         loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
#         loss_disc = (loss_disc_real + loss_disc_fake) / 2
#         disc.zero_grad()
#         loss_disc.backward()
#         opt_disc.step()

#         ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
#         output = disc(fake).reshape(-1)
#         loss_gen = criterion(output, torch.ones_like(output))
#         gen.zero_grad()
#         loss_gen.backward()
#         opt_gen.step()

#         # Print losses occasionally and print to tensorboard
#         if batch_idx % 100 == 0:
#             print(
#                 f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
#                   Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
#             )

#             with torch.no_grad():
#                 fake = gen(fixed_noise)
#                 # take out (up to) 32 examples
#                 img_grid_real = torchvision.utils.make_grid(
#                     real[:BATCH_SIZE], normalize=True
#                 )
#                 img_grid_fake = torchvision.utils.make_grid(
#                     fake[:BATCH_SIZE], normalize=True
#                 )

#                 writer_real.add_image("Real", img_grid_real, global_step=step)
#                 writer_fake.add_image("Fake", img_grid_fake, global_step=step)

#             step += 1

Epoch [0/50] Batch 0/2000                         Loss D: 0.6919, loss G: 0.6963
Epoch [0/50] Batch 100/2000                         Loss D: 0.8040, loss G: 0.5120
Epoch [0/50] Batch 200/2000                         Loss D: 0.9011, loss G: 0.4001
Epoch [0/50] Batch 300/2000                         Loss D: 0.9536, loss G: 0.3507
Epoch [0/50] Batch 400/2000                         Loss D: 0.9797, loss G: 0.3288
Epoch [0/50] Batch 500/2000                         Loss D: 0.9998, loss G: 0.3154
Epoch [0/50] Batch 600/2000                         Loss D: 1.0087, loss G: 0.3062
Epoch [0/50] Batch 700/2000                         Loss D: 1.0171, loss G: 0.3002
Epoch [0/50] Batch 800/2000                         Loss D: 1.0220, loss G: 0.2964
Epoch [0/50] Batch 900/2000                         Loss D: 1.0253, loss G: 0.2938
Epoch [0/50] Batch 1000/2000                         Loss D: 1.0256, loss G: 0.2925
Epoch [0/50] Batch 1100/2000                         Loss D: 1.0255, loss G: 0.2921
Epoc